# Import packages and modules

In [ ]:
import os, glob
import json
import pandas as pd
import numpy as np
import math, random
import datetime as dt
import pickle
from matplotlib import pyplot as plt
import argparse
import path, sys, re, time
from collections import Counter
from scipy.spatial import distance_matrix
from scipy.signal import find_peaks


from module_.info.testbed import activityfiles_new
from module_.info.config import config, feature_name
from module_.readText import create_episodes, time_correction, read_adlmr
from module_.featureExtraction import feature_extraction
from module_.changePointDetection import change_point_detection

In [1]:
from module_.correlation import correlation

result = correlation("hh101","RuLSIF")

/home/kisoo/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/kisoo/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/kisoo/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2493: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


0/79 GA0GB0
1/79 GA0GB1
2/79 GA0GC0
3/79 GA0GC1
4/79 GA0GD0
5/79 GA0GD1
6/79 GA0GE0
7/79 GA0GE1
8/79 GA1GB0
9/79 GA1GB1
10/79 GA1GC0
11/79 GA1GC1
12/79 GA1GD0
13/79 GA1GD1
14/79 GA1GE0
15/79 GA1GE1
16/79 GB0GA0
17/79 GB0GA1
18/79 GB0GC0
19/79 GB0GC1
20/79 GB0GD0
21/79 GB0GD1
22/79 GB0GE0
23/79 GB0GE1
24/79 GB1GA0
25/79 GB1GA1
26/79 GB1GC0
27/79 GB1GC1
28/79 GB1GD0
29/79 GB1GD1
30/79 GB1GE0
31/79 GB1GE1
32/79 GC0GA0
33/79 GC0GA1
34/79 GC0GB0
35/79 GC0GB1
36/79 GC0GD0
37/79 GC0GD1
38/79 GC0GE0
39/79 GC0GE1
40/79 GC1GA0
41/79 GC1GA1
42/79 GC1GB0
43/79 GC1GB1
44/79 GC1GD0
45/79 GC1GD1
46/79 GC1GE0
47/79 GC1GE1
48/79 GD0GA0
49/79 GD0GA1
50/79 GD0GB0
51/79 GD0GB1
52/79 GD0GC0
53/79 GD0GC1
54/79 GD0GE0
55/79 GD0GE1
56/79 GD1GA0
57/79 GD1GA1
58/79 GD1GB0
59/79 GD1GB1
60/79 GD1GC0
61/79 GD1GC1
62/79 GD1GE0
63/79 GD1GE1
64/79 GE0GA0
65/79 GE0GA1
66/79 GE0GB0
67/79 GE0GB1
68/79 GE0GC0
69/79 GE0GC1
70/79 GE0GD0
71/79 GE0GD1
72/79 GE1GA0
73/79 GE1GA1
74/79 GE1GB0
75/79 GE1GB1
76/79 GE1GC0
77/79 GE1

<Figure size 720x72 with 0 Axes>

## testbed

In [ ]:
dir_="dataset/testbed/npy/seminar/MS"
task_dict={i:[np.load("{}/{}".format(dir_, name)) for name in v] for i, v in enumerate(activityfiles_new.values())}
initial_dict={i:k[0] for i, k in enumerate(activityfiles_new.keys())}
label_dict={k[0]:k for k in activityfiles_new.keys()}

episodes, trs, tags = create_episodes(task_dict, initial_dict)
episodes=[time_correction(eps, trs[i]) for i, eps in enumerate(episodes)]

In [ ]:
# view_size = 5

test_idx = 12
test_chunk = episodes[test_idx]
transitions = [trs[test_idx]]

sensor_list = sorted(set(test_chunk[:,0]))

prev_box = {sensor_:None for sensor_ in sensor_list} # Previous state for comparison (If no difference, no update)

features = []

prev_matrix = np.ones((len(sensor_list), len(sensor_list)))
correlation_matrix = np.ones((len(sensor_list), len(sensor_list)))


dict_timestamps={} # Memory of Each sensor's previous timestamp

first_timestamp = float(test_chunk[0][2])-1

l = []
for i, event in enumerate(test_chunk):
    feature = []
    sensor = event[0]
    timestamp = float(event[2])

    if sensor not in dict_timestamps.keys():
        dict_timestamps[sensor]=[first_timestamp, timestamp]
    else:
        dict_timestamps[sensor].append(timestamp)

    index_r = sensor_list.index(sensor) # Current sensor
    sensor_x = dict_timestamps[sensor] # Current sensor's timestamps
    # sensor_x = dict_timestamps[sensor][-view_size:]

    # Update correlation with other sensors
    for sensor_ in dict_timestamps.keys():
        if sensor_==sensor:
            continue
        # sensor_y = dict_timestamps[sensor_][-view_size:]
        sensor_y = dict_timestamps[sensor_] # Other sensor's timestamp

        min_length = min(len(sensor_x), len(sensor_y))

        if min_length == 0: # Not enough data
            continue
        else:
            sensor_x_ = sensor_x[-min_length:]
            sensor_y_ = sensor_y[-min_length:]

        assert len(sensor_x_)==len(sensor_y_)

        index_c = sensor_list.index(sensor_) # The other sensor

        if False:
            correlation_matrix[index_r, index_c] = prev_matrix[index_r, index_c]
            correlation_matrix[index_c, index_r] = prev_matrix[index_r, index_c]
        else:
            correl = np.corrcoef(sensor_x_, sensor_y_)[0,1]#; print(sensor_x_, sensor_y_, sensor, sensor_, correl)
            if not np.isnan(correl): # variance = 0
                correlation_matrix[index_r, index_c]=correl
                correlation_matrix[index_c, index_r]=correl

                prev_matrix[index_r, index_c] = correl
                prev_matrix[index_c, index_r] = correl

                prev_box[sensor_] = sensor_y

    l.append(np.mean(correlation_matrix[index_r]))

print(len(test_chunk), len(l))

fig = plt.figure(figsize=(10, 1))
ax = fig.add_subplot(1, 1, 1)

ax.plot(range(len(l)), l)
for t in transitions:
    plt.axvline(x=t, linestyle=':', color='g')

In [ ]:
window_size = 30
metric = 'RuLSIF'
features = []
for i in range(len(l)):
    bucket=[]
    idx=i-window_size+1
    while idx<=i:
        bucket.append(l[max(0, idx)])
        idx+=1
    
    features.append(bucket)

print(np.array(features).shape)

scores = np.array(change_point_detection(features, metric))
# scores[scores<0] = 0
assert len(scores) == len(l)
plt.bar(range(len(scores)), scores)
for t in transitions:
    plt.axvline(x=t, linestyle=':', color='g')

In [ ]:
ll = []
for i in range(len(l)-1):
    ll.append(abs(l[i]-l[i+1]))
ll = np.array(ll)

plt.bar(range(len(ll)), ll)
for t in transitions:
    plt.axvline(x=t, linestyle=':', color='g')

In [ ]:
sorted(set(test_chunk[:,0]))

## adlmr

In [ ]:
f = open("./dataset/adlmr/annotated", 'rb')
text = f.readlines()
tasks = read_adlmr(text)

## hh

In [ ]:
# load dataset
directory_hh101="./dataset/hh/hh101/ann.txt"

f=open(directory_hh101, 'r')
txt=f.readlines()
events=[]
activity = "Idle"
for i, line in enumerate(txt):
    event = []
    
    try:
        f_info = line.split()
        # Date, time, sensor, value, (label)

        event.append(f_info[2])
        event.append(f_info[3])
        if not ('.' in str(np.array(f_info[0])) + f_info[1]):
            f_info[1] = f_info[1] + '.000000'
        timestamp=datetime.timestamp(datetime.strptime(f_info[0] + f_info[1], "%Y-%m-%d%H:%M:%S.%f"))
        event.append(float(timestamp))           # 3. timestamp

        if len(f_info) != 4: # label
            label = str(' '.join(np.array(f_info[4:])))
            if 'begin' in label:
                activity = label.split('=')[0].strip()
                event.append(activity)
            elif 'end' in label:
                event.append(activity)
                activity = "Idle"
            else:
                event.append(activity)
        else:
            event.append(activity)
        
        events.append(event)

    except:
        print("SKIP")

raw_length = len(events)

print(Counter(np.array(events)[:,3]))

In [ ]:
events = [e for e in events if e[0][0] in list('MD')]

startindices = [i for i in range(1, len(events)) if events[i][3]!=events[i-1][3]]
startindices = [0]+startindices+[len(events)]

dict_consecutiveActivities = {
    'chunks':[],
    'labels':[],
    'lengths':[],
}

chunksize = 8

CoC=[]
CoLbl=[]
CoLen=[]
for i in range(len(startindices)-1):
    start, end_ = startindices[i:i+2]
    chunk=events[start:end_]

    labelset = list(set(np.array(chunk)[:,3]))
    assert len(labelset)==1
    label = labelset[0]

    if len(CoC)==chunksize:
        dict_consecutiveActivities['chunks'].append(CoC)
        dict_consecutiveActivities['labels'].append(CoLbl)
        dict_consecutiveActivities['lengths'].append(CoLen)
        # CoC = CoLbl = CoLen = []
        CoC, CoLbl, CoLen = [], [], []

    CoC.append(chunk)
    CoLbl.append(label)
    CoLen.append(len(chunk))

In [ ]:
"""Test: Second Chunks"""
ti = 3
test_chunk = np.concatenate(dict_consecutiveActivities['chunks'][ti])
test_label = dict_consecutiveActivities['labels'][ti]
test_length = dict_consecutiveActivities['lengths'][ti]

print(test_chunk.shape, test_label, test_length)

transitions = []
start=0
for i in range(len(test_length)-1):
    transitions.append(start+test_length[i])
    start+=test_length[i]


In [ ]:
# window_size = 30
view_size = 5

sensor_list = sorted(set(test_chunk[:,0]))

for sensor in sensor_list:
    print(sensor, sensor_list.index(sensor))

# correlation_matrix = np.diag(np.ones(len(sensor_list)))
correlation_matrix = np.ones((len(sensor_list), len(sensor_list)))
prev1_matrix = prev2_matrix = correlation_matrix
dict_timestamps={}
l = []
r, c = (6, 3)
for i, event in enumerate(test_chunk):
    sensor = event[0]
    timestamp = float(event[2])

    if sensor not in dict_timestamps.keys():
        dict_timestamps[sensor]=[timestamp]
    else:
        dict_timestamps[sensor].append(timestamp)
    index_r = sensor_list.index(sensor)

    sensor_x = dict_timestamps[sensor][-view_size:]
    if len(sensor_x)<view_size:
        l.append(1 if len(l)==0 else l[-1])
        continue

    for sensor_ in dict_timestamps.keys():
        if sensor_==sensor:
            continue
        sensor_y = dict_timestamps[sensor_][-view_size:]
        if len(sensor_y)<view_size:
            continue
        index_c = sensor_list.index(sensor_)
        # col.append(np.corrcoef(sensor_x, sensor_y)[0,1])
        correlation_matrix[index_r, index_c]=np.corrcoef(sensor_x, sensor_y)[0,1]
        correlation_matrix[index_c, index_r]=np.corrcoef(sensor_x, sensor_y)[0,1]
    l.append(np.average(correlation_matrix+prev1_matrix+prev2_matrix)/3)
    prev2_matrix = prev1_matrix
    prev1_matrix = correlation_matrix
    
print(len(test_chunk))
print(len(l))
    
fig = plt.figure(figsize=(10, 1))
ax = fig.add_subplot(1, 1, 1)

ax.plot(range(len(l)), l)
for t in transitions:
    plt.axvline(x=t, linestyle=':', color='g')

## testbed